# **Videos Transcription and Translation with Faster Whisper and ChatGPT**


[![notebook shield](https://img.shields.io/static/v1?label=&message=Notebook&color=blue&style=for-the-badge&logo=googlecolab&link=https://colab.research.google.com/github/lewangdev/autotranslate/blob/main/autotranslate.ipynb)](https://colab.research.google.com/github/lewangdev/autotranslate/blob/main/autotranslate.ipynb)
[![repository shield](https://img.shields.io/static/v1?label=&message=Repository&color=blue&style=for-the-badge&logo=github&link=https://github.com/lewangdev/autotranslate)](https://github.com/lewangdev/autotranslate)

This Notebook will guide you through the transcription and translation of video using [Faster Whisper](https://github.com/guillaumekln/faster-whisper) and ChatGPT. You'll be able to explore most inference parameters or use the Notebook as-is to store the transcript, translation and video audio in your Google Drive.

In [1]:
#@markdown # **Check GPU type** 🕵️

#@markdown The type of GPU you get assigned in your Colab session defined the speed at which the video will be transcribed.
#@markdown The higher the number of floating point operations per second (FLOPS), the faster the transcription.
#@markdown But even the least powerful GPU available in Colab is able to run any Whisper model.
#@markdown Make sure you've selected `GPU` as hardware accelerator for the Notebook (Runtime &rarr; Change runtime type &rarr; Hardware accelerator).

#@markdown |  GPU   |  GPU RAM   | FP32 teraFLOPS |     Availability   |
#@markdown |:------:|:----------:|:--------------:|:------------------:|
#@markdown |  T4    |    16 GB   |       8.1      |         Free       |
#@markdown | P100   |    16 GB   |      10.6      |      Colab Pro     |
#@markdown | V100   |    16 GB   |      15.7      |  Colab Pro (Rare)  |

#@markdown ---
#@markdown **Factory reset your Notebook's runtime if you want to get assigned a new GPU.**

!nvidia-smi -L

!nvidia-smi

GPU 0: Tesla T4 (UUID: GPU-fd914026-b0a9-c87d-346d-f22eb2ebdd25)
Fri Jan 19 08:56:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+--

In [2]:
#@markdown # **Install libraries** 🏗️
#@markdown This cell will take a little while to download several libraries.

#@markdown ---
! pip install faster-whisper==0.10.0
! pip install yt-dlp==2023.11.16
! pip install openai==0.28.1

# Windows Libs：https://github.com/Purfview/whisper-standalone-win/releases/download/libs/cuBLAS.and.cuDNN_win_v2.7z
! apt install -y p7zip-full p7zip-rar
! wget https://github.com/Purfview/whisper-standalone-win/releases/download/libs/cuBLAS.and.cuDNN_linux_v2.7z
! 7z x cuBLAS.and.cuDNN_linux_v2.7z -o/usr/lib



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.9 MB/s eta 0:00:00
  Created wheel for faster-whisper: filename=faster_whisper-0.10.0-py3-none-any.whl size=1539726 sha256=149fdfceb9d2e38d7dc6b68cf321f39d2b33a7c562938048cb383c609997bf37
  Stored in directory: /root/.cache/pip/wheels/b3/4e/9a/bd36d2645cb73f909a3a65a2e317fec5c6a79c8121ab9eb42f
Successfully built faster-whisper
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 23.3 MB/s eta 0:00:00
     ━━━

In [3]:
#@markdown # **Import libraries for Python** 🐍

#@markdown This cell will import all libraries for python code.
import sys
import warnings
from faster_whisper import WhisperModel
from pathlib import Path
import yt_dlp
import subprocess
import torch
import shutil
import numpy as np
from IPython.display import display, Markdown, YouTubeVideo

device = torch.device('cuda:0')
print('Using device:', device, file=sys.stderr)

Using device: cuda:0


In [4]:
#@markdown # **Optional:** Save data in Google Drive 💾
#@markdown Enter a Google Drive path and run this cell if you want to store the results inside Google Drive.

# Uncomment to copy generated images to drive, faster than downloading directly from colab in my experience.
from google.colab import drive
drive_mount_path = Path("/") / "content" / "drive"
drive.mount(str(drive_mount_path))
drive_mount_path /= "My Drive"
#@markdown ---
drive_path = "Colab Notebooks/Videos Transcription and Translation" #@param {type:"string"}
#@markdown ---
#@markdown **Run this cell again if you change your Google Drive path.**

drive_whisper_path = drive_mount_path / Path(drive_path.lstrip("/"))
drive_whisper_path.mkdir(parents=True, exist_ok=True)

Mounted at /content/drive


In [5]:
#@markdown # **Model selection** 🧠

#@markdown As of the first public release, there are 4 pre-trained options to play with:

#@markdown |  Size  | Parameters | English-only model | Multilingual model | Required VRAM | Relative speed |
#@markdown |:------:|:----------:|:------------------:|:------------------:|:-------------:|:--------------:|
#@markdown |  tiny  |    39 M    |     `tiny.en`      |       `tiny`       |     ~0.8 GB     |      ~32x      |
#@markdown |  base  |    74 M    |     `base.en`      |       `base`       |     ~1.0 GB     |      ~16x      |
#@markdown | small  |   244 M    |     `small.en`     |      `small`       |     ~1.4 GB     |      ~6x       |
#@markdown | medium |   769 M    |    `medium.en`     |      `medium`      |     ~2.7 GB     |      ~2x       |
#@markdown | large-v1  |   1550 M   |        N/A         |      `large-v1`       |    ~4.3 GB     |       1x       |
#@markdown | large-v2  |   1550 M   |        N/A         |      `large-v2`       |    ~4.3 GB     |       1x       |
#@markdown | large-v3  |   1550 M   |        N/A         |      `large-v2`       |    ~3.6 GB     |       1x       |

#@markdown ---
model_size = 'large-v2' #@param ['tiny', 'tiny.en', 'base', 'base.en', 'small', 'small.en', 'medium', 'medium.en', 'large-v1', 'large-v2', 'large-v3']
device_type = "cuda" #@param {type:"string"} ['cuda', 'cpu']
compute_type = "float16" #@param {type:"string"} ['float16', 'int8_float16', 'int8']
#@markdown ---
#@markdown **Run this cell again if you change the model.**

model = WhisperModel(model_size, device=device_type, compute_type=compute_type)


config.json:   0%|          | 0.00/2.80k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

In [6]:
#@markdown # **Video selection** 📺

#@markdown Enter the URL of the video you want to transcribe

#@markdown #### **Video or playlist URL**
URL = "https://youtu.be/pTCxXZh6VyE?si=ItBkmwhihuxInLjp" #@param {type:"string"}
# store_audio = True #@param {type:"boolean"}
#@markdown ---
def download_video(URL):
  video_path_local_list = []

  ydl_opts = {
        'format': 'm4a/bestaudio/best',
        'outtmpl': '%(id)s.%(ext)s',
        # ℹ️ See help(yt_dlp.postprocessor) for a list of available Postprocessors and their arguments
        'postprocessors': [{  # Extract audio using ffmpeg
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
        }]
    }

  with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        error_code = ydl.download([URL])
        list_video_info = [ydl.extract_info(URL, download=False)]

  for video_info in list_video_info:
        video_path_local_list.append(Path(f"{video_info['id']}.wav"))
  print(video_path_local_list)
  for video_path_local in video_path_local_list:
      if video_path_local.suffix == ".mp4":
          video_path_local = video_path_local.with_suffix(".wav")
          result  = subprocess.run(["ffmpeg", "-i", str(video_path_local.with_suffix(".mp4")), "-vn", "-acodec", "pcm_s16le", "-ar", "16000", "-ac", "1", str(video_path_local)])
  return video_path_local



In [7]:
def run_model(video_path_local):
  #@markdown # **Run the model** 🚀

  #@markdown Run this cell to execute the transcription of the video. This can take a while and very based on the length of the video and the number of parameters of the model selected above.
  def seconds_to_time_format(s):
      # Convert seconds to hours, minutes, seconds, and milliseconds
      hours = s // 3600
      s %= 3600
      minutes = s // 60
      s %= 60
      seconds = s // 1
      milliseconds = round((s % 1) * 1000)

      # Return the formatted string
      return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d},{int(milliseconds):03d}"


  #@markdown ## **Parameters** ⚙️

  #@markdown ### **Behavior control**
  #@markdown #### Language
  language_options = {
      "Auto Detect": "auto",
      "English": "en",
      "中文(Chinese)": "zh",
      "日本語(Japanese)": "ja",
      "Deutsch(German)": "de",
      "Français(French)": "fr"
  }

  language_option = "Auto Detect" #@param ["Auto Detect", "English", "中文(Chinese)", "日本語(Japanese)", "Deutsch(German)", "Français(French)"] {allow-input: true}
  language = language_options.get(language_option, language_option)

  #@markdown #### initial prompt
  initial_prompt = "Hello, Let's begin to talk." #@param {type:"string"}
  #@markdown ---
  #@markdown #### Word-level timestamps
  word_level_timestamps = True #@param {type:"boolean"}
  #@markdown ---
  #@markdown #### VAD filter
  vad_filter = False #@param {type:"boolean"}
  vad_filter_min_silence_duration_ms = 50 #@param {type:"integer"}
  #@markdown ---


  segments, info = model.transcribe(str(video_path_local), beam_size=5,
                                    language=None if language == "auto" else language,
                                    initial_prompt=initial_prompt,
                                    word_timestamps=word_level_timestamps,
                                    vad_filter=vad_filter,
                                    vad_parameters=dict(min_silence_duration_ms=vad_filter_min_silence_duration_ms))

  language_detected = info.language
  display(Markdown(f"Detected language '{info.language}' with probability {info.language_probability}"))

  fragments = []

  for segment in segments:
    print(f"[{seconds_to_time_format(segment.start)} --> {seconds_to_time_format(segment.end)}] {segment.text}")
    if word_level_timestamps:
      for word in segment.words:
        ts_start = seconds_to_time_format(word.start)
        ts_end = seconds_to_time_format(word.end)
        #print(f"[{ts_start} --> {ts_end}] {word.word}")
        fragments.append(dict(start=word.start,end=word.end,text=word.word))
    else:
      ts_start = seconds_to_time_format(segment.start)
      ts_end = seconds_to_time_format(segment.end)
      #print(f"[{ts_start} --> {ts_end}] {segment.text}")
      fragments.append(dict(start=segment.start,end=segment.end,text=segment.text))

  def merge_word_to_sentences():
    #@title Merge words/segments to sentences

    #@markdown Run this cell to merge words/segments to sentences.
    #@markdown ## **Parameters** ⚙️

    #@markdown ### **Behavior control**
    #@markdown #### Milliseconds gap between_two sentences
    max_gap_ms_between_two_sentence = 200 #@param {type:"integer"}

    import json

    # Merge words/segments to sentences
    def merge_fragments(fragments, gap_ms):
      new_fragments = []
      new_fragment = {}
      length = len(fragments)
      for i, fragment in enumerate(fragments):
        start = fragment['start']
        end = fragment['end']
        text = fragment['text']

        if new_fragment.get('start', None) is None:
          new_fragment['start'] = start
        if new_fragment.get('end', None) is None:
          new_fragment['end'] = end
        if new_fragment.get('text', None) is None:
          new_fragment['text'] = ""

        if start - new_fragment['end'] > gap_ms:
          new_fragments.append(new_fragment)
          new_fragment = dict(start=start, end=end, text=text)
          continue

        new_fragment['end'] = end

        #delimiter = '' if text.startswith('-') else ' '
        delimiter = ' ' if language_detected in ['en', 'de', 'fr'] else ''
        new_fragment['text'] = f"{new_fragment['text']}{delimiter}{text.lstrip()}"

        # End of a sentence when symbols found: [.?]
        if (len(text) > 0 and text[-1] in ['.', '?', '。', '？', '!', '！']) or i == length-1:
          new_fragments.append(new_fragment)
          new_fragment = {}
      return new_fragments


    new_fragments = merge_fragments(fragments, max_gap_ms_between_two_sentence/1000.0)

    # Save as json file
    json_ext_name = ".json"
    json_transcript_file_name = video_path_local.stem + json_ext_name
    with open(json_transcript_file_name, 'w') as f:
      f.write(json.dumps(new_fragments))
    display(Markdown(f"**Transcript SRT file created: {video_path_local.parent / json_transcript_file_name}**"))

    # Save as srt
    srt_ext_name = ".srt"
    srt_transcript_file_name = video_path_local.stem + srt_ext_name
    with open(srt_transcript_file_name, 'w') as f:
      for sentence_idx, fragment in enumerate(new_fragments):
        ts_start = seconds_to_time_format(fragment['start'])
        ts_end = seconds_to_time_format(fragment['end'])
        text = fragment['text']
        print(f"[{ts_start} --> {ts_end}] {text}")
        f.write(f"{sentence_idx + 1}\n")
        f.write(f"{ts_start} --> {ts_end}\n")
        f.write(f"{text.strip()}\n\n")

    try:
      shutil.copy(video_path_local.parent / srt_transcript_file_name,
                drive_whisper_path / srt_transcript_file_name
      )
      display(Markdown(f"**Transcript SRT file created: {drive_whisper_path / srt_transcript_file_name}**"))
      df.at[index, 'status'] = 'Transcribed'
    except:
      display(Markdown(f"**Transcript SRT file created: {video_path_local.parent / srt_transcript_file_name}**"))

  merge_word_to_sentences()


In [23]:
# @markdown # **Mount Drive And Load CSV**
# @markdown You can find path inside drive folder after mounting the drive.
#@markdown CSV File path must be from Google Drive as we have to update it while transcribing.


import pandas as pd
from google.colab import drive
google_drive_csv_path = "/content/drive/MyDrive/video_links.csv" #@param {type:"string"}

# @markdown If CSV file fails it will convert the Video Given above in Video Selection Section. So if you want to transcribe a single video you can just give  a dummy path in  this input.

# Mount Google Drive to save and load checkpoints
drive.mount('/content/drive', force_remount=True)
#Read CSV File
try:
  df = pd.read_csv(google_drive_csv_path)
  print(df)
except:
  video_path = download_video(URL)
  run_model(video_path)

Mounted at /content/drive
                                          video_url status
0  https://youtu.be/pTCxXZh6VyE?si=ItBkmwhihuxInLjp      t
1  https://youtu.be/1aA1WGON49E?si=SI3tzkR6hGYLBxEE      t
2  https://youtu.be/fLeJJPxua3E?si=KF9t4myQjwwTU1Wc      t
3  https://youtu.be/4FDud9Lj5HY?si=jFoMRmLq-0lk_bSO      t
4  https://youtu.be/1j0X9QMF--M?si=CtcYUkWiBuqWMW5g      t
5  https://youtu.be/nyhRNwTfydU?si=609CTnZEnN1Az461      t
6  https://youtu.be/KxvPR0cd3-k?si=b55L145rXy6AVPcj      t
7  https://youtu.be/ry9SYnV3svc?si=70Bp1Ce9eYdcCrh-      t
8  https://youtu.be/YeM5Nw5rEUg?si=mdlFgh1WiypyAaSR      t


In [24]:
# @markdown #Main Cell
# @markdown This code block will run the loop through all the rows in CSV file and transcribe it. Once you run this code block it will transcribe all the videos and mark all the videos converetd s Transcribed so that they won't get transcribed again.
for index, row in df.iterrows():
    video_url = row['video_url']
    status = row['status'].strip().lower()
    print(video_url, "-", status)
    # Check if the video is transcribed
    if status != 'transcribed':
      video_path = download_video(video_url)
      run_model(video_path)
      # Once the video is transcribed, update the status in the CSV file
      df.to_csv('/content/drive/MyDrive/video_links.csv', index=False)




https://youtu.be/pTCxXZh6VyE?si=ItBkmwhihuxInLjp - t
[youtube] Extracting URL: https://youtu.be/pTCxXZh6VyE?si=ItBkmwhihuxInLjp
[youtube] pTCxXZh6VyE: Downloading webpage
[youtube] pTCxXZh6VyE: Downloading ios player API JSON
[youtube] pTCxXZh6VyE: Downloading android player API JSON
[youtube] pTCxXZh6VyE: Downloading m3u8 information
[info] pTCxXZh6VyE: Downloading 1 format(s): 140
[download] Destination: pTCxXZh6VyE.m4a
[download] 100% of  948.19KiB in 00:00:00 at 6.42MiB/s   
[FixupM4a] Correcting container of "pTCxXZh6VyE.m4a"
[ExtractAudio] Destination: pTCxXZh6VyE.wav
Deleting original file pTCxXZh6VyE.m4a (pass -k to keep)
[youtube] Extracting URL: https://youtu.be/pTCxXZh6VyE?si=ItBkmwhihuxInLjp
[youtube] pTCxXZh6VyE: Downloading webpage
[youtube] pTCxXZh6VyE: Downloading ios player API JSON
[youtube] pTCxXZh6VyE: Downloading android player API JSON
[youtube] pTCxXZh6VyE: Downloading m3u8 information
[PosixPath('pTCxXZh6VyE.wav')]


Detected language 'en' with probability 0.99560546875

[00:00:01,020 --> 00:00:02,960]  Welcome to English in a Minute.
[00:00:03,580 --> 00:00:06,440]  Most people enjoy going to parties.
[00:00:07,280 --> 00:00:11,580]  You get to be around friends, eat food, and maybe listen to music.
[00:00:12,180 --> 00:00:14,880]  But do your parties ever have animals?
[00:00:15,800 --> 00:00:16,760]  Party animal.
[00:00:17,620 --> 00:00:21,100]  Hey Jonathan, are you as ready as I am for the weekend?
[00:00:21,420 --> 00:00:22,360]  Oh yeah.
[00:00:22,840 --> 00:00:27,660]  I'm going to my brother's birthday party on Friday, my cousin is having her graduation
[00:00:27,660 --> 00:00:32,360]  party Saturday, and I'm hosting a karaoke party on Sunday.
[00:00:32,760 --> 00:00:33,180]  Wow!
[00:00:33,660 --> 00:00:36,580]  You are quite the party animal, aren't you?
[00:00:37,660 --> 00:00:38,120]  That's nothing.
[00:00:38,460 --> 00:00:39,940]  You should have seen me in college.
[00:00:41,140 --> 00:00:44,520]  A party animal is not an actual animal

**Transcript SRT file created: pTCxXZh6VyE.json**

[00:00:01,020 --> 00:00:02,960]  Welcome to English in a Minute.
[00:00:03,580 --> 00:00:06,440]  Most people enjoy going to parties.
[00:00:07,280 --> 00:00:09,860]  You get to be around friends, eat food,
[00:00:10,100 --> 00:00:11,580]  and maybe listen to music.
[00:00:12,180 --> 00:00:14,880]  But do your parties ever have animals?
[00:00:15,800 --> 00:00:16,760]  Party animal.
[00:00:17,620 --> 00:00:18,280]  Hey Jonathan,
[00:00:18,780 --> 00:00:21,100]  are you as ready as I am for the weekend?
[00:00:21,420 --> 00:00:22,360]  Oh yeah.
[00:00:22,840 --> 00:00:25,580]  I'm going to my brother's birthday party on Friday,
[00:00:26,060 --> 00:00:28,580]  my cousin is having her graduation party Saturday,
[00:00:28,940 --> 00:00:32,360]  and I'm hosting a karaoke party on Sunday.
[00:00:32,760 --> 00:00:33,180]  Wow!
[00:00:33,660 --> 00:00:36,580]  You are quite the party animal, aren't you?
[00:00:37,400 --> 00:00:38,120]  That's nothing.
[00:00:38,460 --> 00:00:39,940]  You shou

...

Detected language 'en' with probability 0.98095703125

[00:00:13,600 --> 00:00:22,160]  When I asked my father to write a short and sweet speech on the subject Discipline, he
[00:00:22,160 --> 00:00:23,240]  was so happy.
[00:00:23,240 --> 00:00:29,120]  It seems he was waiting for an opportunity to teach me some Discipline.
[00:00:29,740 --> 00:00:36,100]  With a smile, he asked me to start the speech, with a parody towards the Bible.
[00:00:37,760 --> 00:00:40,140]  I am not worthy to speak about Discipline.
[00:00:41,140 --> 00:00:44,220]  By saying some words, I will learn better.
[00:00:45,880 --> 00:00:46,860]  What is Discipline?
[00:00:47,860 --> 00:00:52,280]  Discipline is nothing but the training people to obey rules.
[00:00:53,200 --> 00:00:59,840]  It's the tradition to train people to obey rules, especially in a nation like India.
[00:01:00,800 --> 00:01:06,440]  Without having a disciplined life, we cannot work towards our goals.
[00:01:07,300 --> 00:01:12,580]  This is the reason why its importance is taught right from our 

**Transcript SRT file created: YeM5Nw5rEUg.json**

[00:00:13,600 --> 00:00:21,140]  When I asked my father to write a short and sweet speech on the subject Discipline,
[00:00:22,000 --> 00:00:23,240]  he was so happy.
[00:00:23,240 --> 00:00:29,120]  It seems he was waiting for an opportunity to teach me some Discipline.
[00:00:29,740 --> 00:00:30,780]  With a smile,
[00:00:31,120 --> 00:00:32,780]  he asked me to start the speech,
[00:00:33,700 --> 00:00:36,100]  with a parody towards the Bible.
[00:00:37,570 --> 00:00:40,140]  I am not worthy to speak about Discipline.
[00:00:41,140 --> 00:00:42,680]  By saying some words,
[00:00:43,040 --> 00:00:44,220]  I will learn better.
[00:00:45,610 --> 00:00:46,860]  What is Discipline?
[00:00:47,860 --> 00:00:52,280]  Discipline is nothing but the training people to obey rules.
[00:00:53,200 --> 00:00:57,120]  It's the tradition to train people to obey rules,
[00:00:57,600 --> 00:00:59,840]  especially in a nation like India.
[00:01:00,800 --> 00:01:03,780]  Without having a disciplined life

**Transcript SRT file created: /content/drive/My Drive/Colab Notebooks/Videos Transcription and Translation/YeM5Nw5rEUg.srt**